In [1]:
%pip install denoising_diffusion_pytorch


You should consider upgrading via the 'c:\Users\attan\.pyenv\pyenv-win\versions\3.8.10\python.exe -m pip install --upgrade pip' command.


In [7]:
import torch
%pip uninstall numpy
!py -m pip install numpy==1.23.1 --user

# import gaussian diffusion from denoising-diffusion-pytorch

# from denoising_diffusion_pytorch import Unet, GaussianDiffusion

In [ ]:
# Create a UNet model
model = Unet(
    dim = 64,
    dim_mults = (1, 2, 4, 8)
)

# Create a Gaussian Diffusion model
diffusion = GaussianDiffusion(
    model,
    image_size = 128,
    timesteps = 100,   # number of steps
    loss_type = 'l1'    # L1 or L2
)

trainer = Trainer(
    diffusion,
    r'C:\Users/\attan\Downloads\ml\target_dir\train',
    train_batch_size = 32,
    train_lr = 2e-5,
    train_num_steps = 700000,         # total training steps
    gradient_accumulate_every = 2,    # gradient accumulation steps
    ema_decay = 0.995,                # exponential moving average decay
    amp = True                        # turn on mixed precision
)

training_images = torch.rand(8, 3, 128, 128) # images are normalized from 0 to 1
loss = diffusion(training_images)
loss.backward()
# after a lot of training

sampled_images = diffusion.sample(batch_size = 4)
sampled_images.shape # (4, 3, 128, 128)


NameError: name 'Unet' is not defined